In [ ]:
import numpy as np
import scipy 
import skimage
import matplotlib.pyplot as plt
import cv2

In [ ]:
from matplotlib.patches import Circle
def show_all_circles(image, cx, cy, sig, color='r'):
    """
    image: numpy array, representing the grayscsale image
    cx, cy: numpy arrays or lists, centers of the detected blobs
    rad: numpy array or list, radius of the detected blobs
    """
    fig, ax = plt.subplots()
    ax.set_aspect('equal')
    ax.imshow(image, cmap='gray')
    for x, y, s in zip(cx, cy, sig):
        circ = Circle((x, y), s*(2**0.5), color=color, fill=False)
        ax.add_patch(circ)

    plt.title('%i circles' % len(cx))
    plt.show()

In [ ]:
img_dir = "data/"
img_list = ["butterfly.jpg","einstein.jpg","sunflowers.jpg","fishes.jpg"]

In [ ]:
for loc in img_list:
    img = cv2.imread(img_dir+loc,cv2.IMREAD_GRAYSCALE)
    img/= 255
    